In [1]:
import pandas as pd
import geojson
from datetime import datetime, timedelta

from corncast import Location, corn_forecast, make_obs_df, make_forecast_df

In [2]:
locs = [Location("Mt. Rose Summit", 39.3144, -119.9173),
        Location("Berkeley", 37.8715, -122.27),
        Location("Mt. Shasta - Bunny Flat", 41.353, -122.234)]
locations = {l.name : l for l in locs}

In [3]:
(obs_df, comb_6h_df) = corn_forecast(rose)

Calling: /points/39.3144,-119.9173
Calling: https://api.weather.gov/gridpoints/REV/40,97/stations
Calling: /stations/SLIDE/observations?start=2023-11-28T20%3A58%3A11Z&end=2023-12-03T20%3A58%3A11Z


/home/smerdis/Dropbox/code/corncast/corncast/corncast.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  obs_df_full = pd.concat(


Calling: /points/39.3144,-119.9173
Calling: https://api.weather.gov/gridpoints/REV/40,97/forecast/hourly


In [4]:
def highlight(row):
    if row['cycle'] and row['obs_precip_iszero']:
        return ['background-color: green'] * len(row)
    else:
        return ['background-color: white'] * len(row)

fmts = {'date':lambda x: f"{x:%m-%d}",
        'obs_precip': lambda x: f"{x:.1f}"}
out = comb_6h_df.style.apply(highlight, axis=1)
out.format(fmts).hide(subset=['obs_precip_iszero'], axis=1)

,date,cycle,obs_precip,prob_precip
0,11-28,True,0.0,nan
1,11-29,False,0.0,nan
2,11-30,False,0.0,nan
3,12-01,False,0.0,nan
4,12-02,False,0.0,nan
5,12-03,False,0.0,nan
6,12-04,True,nan,14.000000
7,12-05,True,nan,3.000000
8,12-06,False,nan,57.000000
9,12-07,True,nan,64.000000


In [5]:
# set up some variables
now = datetime.now()
obs_period = timedelta(days=5)
start = now-obs_period
end = now
tcol = 'tempF'
foo = pd.concat([pd.json_normalize(o) for o in rose.get_obs(start, end)], ignore_index=True)

Calling: /points/39.3144,-119.9173
Calling: https://api.weather.gov/gridpoints/REV/40,97/stations
Calling: /stations/SLIDE/observations?start=2023-11-28T20%3A58%3A14Z&end=2023-12-03T20%3A58%3A14Z


/tmp/ipykernel_19442/1239433699.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  foo = pd.concat([pd.json_normalize(o) for o in rose.get_obs(start, end)], ignore_index=True)


In [6]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
#import pandas as pd

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Corn Forecast', style={'textAlign':'center'}),
    dcc.Dropdown(list(locations), next(iter(locations)), id='loc-selection'),
    dcc.Graph(id='obs-temp'),
    dcc.Graph(id='fcst-temp')
])

@callback(
    Output('obs-temp', 'figure'),
    Input('loc-selection', 'value')
)
def update_obs(value, tcol='tempF'):
    now = datetime.now()
    obs_period = timedelta(days=5)
    start = now-obs_period
    end = now
    obs_df = make_obs_df(locations[value], start, end)
    hour_means = obs_df.groupby(["datehour"])[tcol].mean().reset_index()
    return px.line(hour_means, x='datehour', y=tcol)

@callback(
    Output('fcst-temp', 'figure'),
    Input('loc-selection', 'value')
)
def update_fcst(value, tcol='tempF'):
    fcst_df = make_forecast_df(locations[value])
    return px.line(fcst_df, x='startTime', y=tcol)

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/
